<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#MantisShrimp" data-toc-modified-id="MantisShrimp-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>MantisShrimp</a></span><ul class="toc-item"><li><span><a href="#Install" data-toc-modified-id="Install-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Install</a></span></li><li><span><a href="#How-to-use" data-toc-modified-id="How-to-use-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>How to use</a></span></li></ul></li></ul></div>

# MantisShrimp

> Built on top of [pytorch-lightining](https://github.com/PyTorchLightning/pytorch-lightning), `MantisShrimp` is an object detection framework focused on application.

MantisShrimp is in very early development, all contributions are welcome!

Be sure to check the `issues` board =)

## Install

```python
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
pip install mantisshrimp
```

## How to use

Check the `examples` folder for a ton of usage cases. It should be very easy to get started.